# I will create a Linear Regression model

In [1]:
# Import the necessary libraries 
import numpy as np
import pandas as pd 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

In [2]:

# Read train features
train_aug = pd.read_csv("../Data/train_labels_augmented.csv")
train_aug.set_index(train_aug.sample_id, inplace=True)
train_aug


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,...,3_ratio_peak_abund,3_ratio_meanstd_abund,4_ratio,4_ratio_peak_temp,4_ratio_peak_abund,4_ratio_meanstd_abund,5_ratio,5_ratio_peak_temp,5_ratio_peak_abund,5_ratio_meanstd_abund
sample_id,,,,,,,,,,,,,,,,,,,,,
S0000,S0000,0,0,0,0,0,0,0,0,1,...,0.024563,0.009736,28.0,35.420,0.010877,0.009101,19.0,105.458,0.006600,0.002841
S0001,S0001,0,1,0,0,0,0,0,0,0,...,0.033209,0.024006,16.0,497.404,0.093082,0.020585,18.0,-60.370,0.030234,0.019214
S0002,S0002,0,0,0,0,0,1,0,0,0,...,0.294738,0.075850,16.0,545.304,0.040446,0.013012,1.0,145.673,0.004003,0.001801
S0003,S0003,0,1,0,1,0,0,0,0,1,...,0.260027,0.061956,28.0,593.222,0.081701,0.031606,16.0,593.222,0.083077,0.028375
S0004,S0004,0,0,0,1,0,1,1,0,0,...,0.238290,0.082294,16.0,471.733,0.084732,0.026218,44.0,333.273,0.029998,0.013378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S0749,S0749,0,0,0,0,0,0,0,0,0,...,0.270481,0.187613,28.0,523.057,0.130122,0.079981,16.0,228.295,0.082464,0.057972
S0750,S0750,0,0,0,0,0,0,1,0,0,...,0.403334,0.152057,32.0,906.311,0.158759,0.122547,16.0,664.846,0.079410,0.041182
S0751,S0751,0,0,0,0,0,0,0,1,0,...,0.261788,0.143669,28.0,-60.026,0.065964,0.068941,32.0,96.019,0.032240,0.019636


In [3]:
temp_bins = pd.read_csv("../Features Engineering/train_features_temp_bin_PCA.csv")
temp_bins.set_index(train_aug.sample_id, inplace=True)
temp_bins

,Unnamed: 0,0,1,2,0.1,1.1,2.1,0.2,1.2,2.2,...,2.12,0.13,1.13,2.13,0.14,1.14,2.14,0.15,1.15,2.15
sample_id,,,,,,,,,,,,,,,,,,,,,
S0000,0,-0.621492,0.165280,0.125295,-0.784563,-1.268913,0.147361,-0.850240,-1.674688,0.228735,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15
S0001,1,2.007919,-3.813076,-0.893565,-0.771116,-1.316065,-0.368455,-0.866575,-1.992731,0.967201,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15
S0002,2,-0.621492,0.165280,0.125295,-0.757530,-0.833097,1.047529,-0.839254,-1.120552,-0.780532,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15
S0003,3,-0.621492,0.165280,0.125295,-0.794925,-1.349837,-0.085693,-0.837493,-1.259507,-0.268338,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15
S0004,4,-0.621492,0.165280,0.125295,-0.782964,-1.322934,-0.275282,-0.785496,-0.772100,-0.888278,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S0749,749,-0.621492,0.165280,0.125295,-0.162081,0.546671,0.342431,-0.107018,1.277499,-0.871815,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15
S0750,750,-0.621492,0.165280,0.125295,-0.775474,-1.443004,-0.449175,-0.838226,-1.865213,0.726747,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15
S0751,751,1.185711,-2.569778,-3.077286,-0.513721,2.316029,7.273461,-0.732185,-0.011605,-0.953857,...,-0.09684,-0.983755,-0.177357,-0.178857,-0.559546,-0.15498,-0.039571,-0.362594,2.268347e-15,1.297573e-15


### I perform a train/test split on our overall training dataset and separate the predictors (columns after the ions) and the predictions (columns of ions). Then I create a KNN Classification model that takes into consideration the 3 closest neighbors and uses them to predict whether the ions are present.

In [4]:
# Separate inputs from outputs
X = temp_bins.iloc[:,1:49]
y = train_aug.iloc[:,1:11]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression().fit(X_train, y_train)

y_pred = reg.predict(X_test)

### Our predictions for the test set come out as an array, so we need to put it in a dataframe and put back the column labels.

In [5]:
y_pred_df=pd.DataFrame(y_pred, columns=['basalt','carbonate','chloride','iron_oxide','oxalate','oxychlorine','phyllosilicate','silicate','sulfate','sulfide'])
y_pred_df

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
0,0.250225,0.100374,-0.090815,0.134457,-0.039034,0.077420,0.252659,0.269037,0.038796,0.039781
1,0.088341,0.219707,0.080064,-0.080790,0.213968,0.185345,0.257558,0.094911,0.229111,0.002373
2,0.095960,0.102619,0.165617,0.131979,0.072709,0.304102,0.326315,0.036568,0.334034,0.065164
3,0.082676,0.035780,0.269614,0.081394,0.021695,0.156141,0.307211,0.059353,0.282110,0.081637
4,0.218629,0.160184,0.051717,0.586442,-0.019398,0.442490,0.437825,0.140136,0.268757,0.022467
...,...,...,...,...,...,...,...,...,...,...
146,0.146468,0.066777,0.173437,0.325075,-0.075481,0.265335,0.161292,0.088566,0.241849,0.241342
147,-0.228923,-0.328673,-0.059398,-0.263546,0.125373,-0.087535,0.674828,-0.187830,-0.175914,0.068439
148,-0.016640,0.124002,0.230152,0.195972,0.019815,0.119044,0.289937,0.059878,0.171392,0.032587
149,0.178589,0.166753,0.120624,0.364670,0.022467,0.279790,0.459014,0.155057,0.213098,-0.041634


### Comparing what we got with what we expected to get.

In [6]:
y_test=y_test.reset_index()
y_test.drop('sample_id', axis=1, inplace=True)
y_test_int=y_test.replace(0,"0").replace(1,"1")
y_test

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,1,1,0,1,0,1,1,1,1,0
3,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
146,0,0,0,0,0,1,0,0,0,1
147,0,0,0,0,0,0,1,0,0,0
148,0,0,0,0,0,1,0,0,0,0
149,1,1,0,1,0,1,1,1,1,0


### Might be worth rounding to make the model easier to analyze.

In [7]:
y_pred_integers = y_pred_df.round(0)
y_pred_integers= y_pred_integers.replace(-0.0,"0").replace(0.0,"0").replace(1.0,"1")
y_pred_df = y_pred_integers
y_pred_df

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
146,0,0,0,0,0,0,0,0,0,0
147,0,0,0,0,0,0,1,0,0,0
148,0,0,0,0,0,0,0,0,0,0
149,0,0,0,0,0,0,0,0,0,0


### I think we can already tell this is going to be bad, but we can see how many rows of our predictions were identical to the corresponding row of our expectation.

In [8]:
count=0
for i in range(len(y_test_int)-1):
    if y_test_int.iloc[i].equals(y_pred_integers.iloc[i]):
        count=count+1

### Next we calculate the percent accuracy for each sample. (Just check whether we got all columns correct for each sample using this model.)

In [9]:
count/(len(y_test)-1)*100

20.0

### 19% accuracy is not great, but we should not be surprised with a linear model. Also that's for the entire row being correct. Let's try to examine how good this model is for the presence of each ion separately.

#### Basalt:

In [10]:
basaltcount=0
for i in range(len(y_test)-1):
    if y_test_int.basalt[i]==(y_pred_df.basalt[i]):
        basaltcount=basaltcount+1
basalt = basaltcount/len(y_test-1)*100
basalt

86.75496688741721

#### Carbonate

In [11]:
carbonatecount=0
for i in range(len(y_test)-1):
    if y_test_int.carbonate[i]==(y_pred_df.carbonate[i]):
        carbonatecount=carbonatecount+1
carbonate = carbonatecount/len(y_test-1)*100
carbonate

88.74172185430463

#### Chloride

In [12]:
chloridecount=0
for i in range(len(y_test)-1):
    if y_test_int.chloride[i]==(y_pred_df.chloride[i]):
        chloridecount=chloridecount+1
chloride = chloridecount/len(y_test-1)*100
chloride

86.09271523178808

#### Iron Oxide

In [13]:
IOcount=0
for i in range(len(y_test)-1):
    if y_test_int.iron_oxide[i]==(y_pred_df.iron_oxide[i]):
        IOcount=IOcount+1
ironoxide = IOcount/len(y_test-1)*100
ironoxide

83.44370860927152

#### Oxalate

In [14]:
Oxcount=0
for i in range(len(y_test)-1):
    if y_test_int.oxalate[i]==(y_pred_df.oxalate[i]):
        Oxcount=Oxcount+1
oxalate = Oxcount/len(y_test-1)*100
oxalate

92.71523178807946

#### Oxychlorine

In [15]:
Oxycount=0
for i in range(len(y_test)-1):
    if y_test_int.oxychlorine[i]==(y_pred_df.oxychlorine[i]):
        Oxycount=Oxycount+1
oxychlorine = Oxycount/len(y_test-1)*100
oxychlorine

75.49668874172185

#### Phyllosilicate

In [16]:
Phycount=0
for i in range(len(y_test)-1):
    if y_test_int.phyllosilicate[i]==(y_pred_df.phyllosilicate[i]):
        Phycount=Phycount+1
phyllosilicate = Phycount/len(y_test-1)*100
phyllosilicate

71.52317880794702

#### Silicate

In [17]:
Silcount=0
for i in range(len(y_test)-1):
    if y_test_int.silicate[i]==(y_pred_df.silicate[i]):
        Silcount=Silcount+1
silicate = Silcount/len(y_test-1)*100
silicate

86.09271523178808

#### Sulfate

In [18]:
Sulfatecount=0
for i in range(len(y_test)-1):
    if y_test_int.sulfate[i]==(y_pred_df.sulfate[i]):
        Sulfatecount=Sulfatecount+1
sulfate = Sulfatecount/len(y_test-1)*100
sulfate

76.82119205298014

#### Sulfide

In [19]:
Sulfidecount=0
for i in range(len(y_test)-1):
    if y_test_int.sulfide[i]==(y_pred_df.sulfide[i]):
        Sulfidecount=Sulfidecount+1
sulfide = Sulfidecount/len(y_test-1)*100
sulfide

96.02649006622516

### Now the average of these percent accuracies is slightly better.

In [20]:
(basalt + carbonate + chloride + ironoxide + oxalate + oxychlorine + phyllosilicate + silicate + sulfate + sulfide)/10

84.37086092715231

### Honestly, much better than I thought we'd do.